In [ ]:
!pip install -qU xpander-sdk llama-index llama-index-llms-openai python-dotenv openai


In [4]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from os import environ
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]
XPANDER_API_KEY = environ["XPANDER_API_KEY"]
XPANDER_API_KEY_MULTI_AGENT = environ["XPANDER_API_KEY_MULTI_AGENT"]
XPANDER_AGENT_ID = environ["XPANDER_AGENT_ID"]
XPANDER_AGENT_ID_MULTI_AGENT = environ["XPANDER_AGENT_ID_MULTI_AGENT"]

In [5]:
llm = OpenAI(model="gpt-4o")
llama_agent = ReActAgent.from_llm(llm=llm, verbose=True)
llama_agent.chat("What is the weather in Tokyo?")

Added user message to memory: What is the weather in Tokyo?


AgentChatResponse(response="I'm unable to provide real-time weather updates. For the current weather in Tokyo, I recommend checking a reliable weather website or using a weather app.", sources=[], source_nodes=[], is_dummy_stream=False, metadata=None)

In [2]:
from openai import OpenAI
from xpander_sdk import XpanderClient
openai_client = OpenAI(api_key=OPENAI_API_KEY)

xpander_client = XpanderClient(api_key=XPANDER_API_KEY_MULTI_AGENT)
xpander_agent = xpander_client.agents.get(agent_id=XPANDER_AGENT_ID_MULTI_AGENT)


xpander_agent.add_task("Summarize news from the last 24 hours and provide a list of the most important ones, also find the profiles of the relevant people in Linkedin if needed")
step = 1
while not xpander_agent.is_finished():
    print("-"*100)
    print(f"Step {step}")
    response = openai_client.chat.completions.create(
                model= 'gpt-4o',
                messages=xpander_agent.memory.retrieve_messages(),
                tools=xpander_agent.get_tools(),
                tool_choice="auto",
                temperature=0.0
        )
            
    xpander_agent.memory.add_messages(response.model_dump())
    print("model response", response.model_dump())
    tool_calls = XpanderClient.extract_tool_calls(llm_response=response.model_dump())
    for tool_call in tool_calls:
        print("tool name: ", tool_call.name)
        print("tool call generated payload: ", tool_call.payload)
    tool_responses = xpander_agent.run_tools(tool_calls=tool_calls)
    for tool_response in tool_responses:
        print("tool name: ", tool_response.function_name)
        print("tool status code: ", tool_response.status_code)
        print("tool result: ", tool_response.result)
    step += 1

b'{"stdout":"bG9hZGluZyBhZ2VudCBmOTMxYTliNy02YzAzLTRkMzktOGQ4Yi0wOWE1M2I0Yzc4YTMK"}\n'


----------------------------------------------------------------------------------------------------
Step 1
model response {'id': 'chatcmpl-AwedpeRvF9GpCFmBbL0xqTi1jyozf', 'choices': [{'finish_reason': 'tool_calls', 'index': 0, 'logprobs': None, 'message': {'content': None, 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'call_3DvPhtHTZD7tzFp7jW61o8cu', 'function': {'arguments': '{"bodyParams":{"input_task":"Summarize news from the last 24 hours and provide a list of the most important ones."},"queryParams":{},"pathParams":{}}', 'name': 'HackerNewsProcessSubTaskWithInsights'}, 'type': 'function'}]}}], 'created': 1738542613, 'model': 'gpt-4o-2024-08-06', 'object': 'chat.completion', 'service_tier': 'default', 'system_fingerprint': 'fp_4691090a87', 'usage': {'completion_tokens': 54, 'prompt_tokens': 1018, 'total_tokens': 1072, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejecte

b'{"stdout":"cnVubmluZyB0b29sIEhhY2tlck5ld3NQcm9jZXNzU3ViVGFza1dpdGhJbnNpZ2h0cyBvbiBhZ2VudCBmOTMxYTliNy02YzAzLTRkMzktOGQ4Yi0wOWE1M2I0Yzc4YTMgd2l0aCBleGVjdXRpb24gMTU0NDc5ZDMtMDM2NS00YmVkLTljMDQtYmU4Yzg1NzFiZjIwCg=="}\n'


tool name:  HackerNewsProcessSubTaskWithInsights
tool status code:  200
tool result:  OK
----------------------------------------------------------------------------------------------------
Step 2
model response {'id': 'chatcmpl-AwedurW96mGrYCKX0T2aZnHXjlcjr', 'choices': [{'finish_reason': 'tool_calls', 'index': 0, 'logprobs': None, 'message': {'content': None, 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'call_mhCHCoL4zKrYp1Sjzl2XjmtT', 'function': {'arguments': '{"bodyParams":{"result":"The news from the last 24 hours has been summarized and the most important ones have been listed. If you need profiles of relevant people on LinkedIn, please specify the individuals or topics of interest.","is_success":true},"queryParams":{},"pathParams":{}}', 'name': 'xpfinish-agent-execution-finished'}, 'type': 'function'}]}}], 'created': 1738542618, 'model': 'gpt-4o-2024-08-06', 'object': 'chat.completion', 'service_tier': 'default', 'system_fing

b'{"stdout":"cnVubmluZyB0b29sIHhwZmluaXNoLWFnZW50LWV4ZWN1dGlvbi1maW5pc2hlZCBvbiBhZ2VudCBmOTMxYTliNy02YzAzLTRkMzktOGQ4Yi0wOWE1M2I0Yzc4YTMgd2l0aCBleGVjdXRpb24gMTU0NDc5ZDMtMDM2NS00YmVkLTljMDQtYmU4Yzg1NzFiZjIwCg=="}\n'


...

tool name:  HackerNewsItemsRetrieveItemDetailsById
tool status code:  200
tool result:  {'by': 'MilnerRoute', 'descendants': 0, 'id': 42913111, 'score': 4, 'time': 1738539977, 'title': "OpenAI makes surprise livestream today for 'Deep Research' announcement", 'type': 'story', 'url': 'https://slashdot.org/story/25/02/02/2342245/openai-makes-surprise-livestream-today-for-deep-research-announcement'}
tool name:  HackerNewsItemsRetrieveItemDetailsById
tool status code:  200
tool result:  {'by': 'shayarma', 'descendants': 0, 'id': 42913110, 'score': 1, 'text': 'They even have Typescript libraries. Crazy.', 'time': 1738539963, 'title': 'Show HN: A full on end to end Payment System', 'type': 'story', 'url': 'https://stripe.com/'}
tool name:  HackerNewsItemsRetrieveItemDetailsById
tool status code:  200
tool result:  {'by': 'walterbell', 'descendants': 8, 'id': 42913104, 'kids': [42913410, 42913408, 42913392, 42913376, 42913294, 42913380], 'score': 14, 'time': 1738539914, 'title': 'Solar Plant

b'{"stdout":"cnVubmluZyB0b29sIHhwZmluaXNoLWFnZW50LWV4ZWN1dGlvbi1maW5pc2hlZCBvbiBhZ2VudCBkY2NhY2I0Ny1mOTBmLTRlOTYtYmYyZi1hOTI1ODZiZGM2NDEgd2l0aCBleGVjdXRpb24gMGViZTE2M2EtNzM0NS00ZWRiLTk4YTItZjIyMzE5NTQzODhhCg=="}\n'


tool name:  xpfinish-agent-execution-finished
tool status code:  200
tool result:  OK


b'{"stdout":"c3dpdGNoaW5nIGZyb20gZXhlY3V0aW9uIDBlYmUxNjNhLTczNDUtNGVkYi05OGEyLWYyMjMxOTU0Mzg4YSB0byBwYXJlbnQgZXhlY3V0aW9uIDE1NDQ3OWQzLTAzNjUtNGJlZC05YzA0LWJlOGM4NTcxYmYyMAo="}\n'


----------------------------------------------------------------------------------------------------
Step 11
model response {'id': 'chatcmpl-AwegjgEb4qhHrQGvkkIS5WenuaPDC', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': "Here are some of the most interesting news stories from the last 24 hours on Hacker News:\n\n1. **Federal government grant award search**\n   - [Read more](https://datarepublican.com/award_search/)\n\n2. **Trump's tariffs on Taiwan's chips likely to backfire**\n   - [Read more](https://focustaiwan.tw/business/202502020007)\n\n3. **What's Happening to Firefox**\n   - [Read more](https://twitter.com/Pirat_Nation/status/1885871258270851135)\n\n4. **Build systems and their effects on versioning and API changes**\n   - [Read more](https://utcc.utoronto.ca/~cks/space/blog/programming/BuildSystemsAndAPIChanges)\n\n5. **Nasdaq futures down, oil up as tariffs loom with largest US trading partners**\n   - [Read more](https://www.cmegro

In [6]:
print(xpander_agent.retrieve_execution_result().result)